Read all the data from the api into a list

In [ ]:
import requests
import pandas as pd

#function to retrieve all goal locations from a team in a given season
def getTeamGoals(season):
    goalLocations = []
    #number of games in a season varies by year
    season = int(season) #cast to an int
    
    if season>2020: seasonLength = str(41*32)
    elif season == 2020: seasonLength = str(28*31)
    elif season < 2020 and season >= 2017: seasonLength = str(41*31)
    elif season < 2017 and season >= 2010: seasonLength = str(41*30)
    else: raise Exception("There is no location data before 2010")
    #get start and end gameIds
    season = str(season) #cast back to string
    idStart = int(season+'02'+'0001')
    idEnd = int(season+'02'+seasonLength)

    #loop through every game of the season
    for gameId in range(idStart,idEnd):
        #get url with proper gameId, convert to json
        API_URL = f"https://statsapi.web.nhl.com/api/v1/game/{gameId}/feed/live"
        shotR = requests.get(API_URL, params={"Content-Type": "application/json"})
        shotData = shotR.json()

        #filter goals
        goals = filter(lambda a : (a["result"]["event"] == "Goal"),shotData["liveData"]["plays"]["allPlays"]) 

        #add these goals to list of locations, along with who scored them
        for goal in goals:
            g = goal["coordinates"]
            g['player'] = goal['players'][0]['player']['fullName']
            #need to check if it exists, as some goals do not have this type
            if goal['result'].get('secondaryType'):
                g['secondaryType'] = goal['result']['secondaryType']
            g['team'] = goal['team']['name']
            g['strength'] = goal['result']['strength']['name']
            g['period'] = goal['about']['period']
            goalLocations.append(g)
    #return a list of locations
    return goalLocations

In [42]:
%%time
seasonGoals = getTeamGoals('2018')

Wall time: 3min 31s


Preview of the list

In [43]:
seasonGoals

[{'x': 87.0,
  'y': -3.0,
  'player': 'Artturi Lehkonen',
  'secondaryType': 'Wrap-around',
  'team': 'Montréal Canadiens',
  'strength': 'Even',
  'period': 1},
 {'x': -64.0,
  'y': -14.0,
  'player': 'Auston Matthews',
  'secondaryType': 'Snap Shot',
  'team': 'Toronto Maple Leafs',
  'strength': 'Power Play',
  'period': 1},
 {'x': 63.0,
  'y': 5.0,
  'player': 'John Tavares',
  'secondaryType': 'Snap Shot',
  'team': 'Toronto Maple Leafs',
  'strength': 'Even',
  'period': 2},
 {'x': -53.0,
  'y': 7.0,
  'player': 'Andrew Shaw',
  'secondaryType': 'Slap Shot',
  'team': 'Montréal Canadiens',
  'strength': 'Power Play',
  'period': 2},
 {'x': 74.0,
  'y': 4.0,
  'player': 'Auston Matthews',
  'secondaryType': 'Snap Shot',
  'team': 'Toronto Maple Leafs',
  'strength': 'Even',
  'period': 4},
 {'x': -76.0,
  'y': -29.0,
  'player': 'T.J. Oshie',
  'secondaryType': 'Snap Shot',
  'team': 'Washington Capitals',
  'strength': 'Even',
  'period': 1},
 {'x': -84.0,
  'y': -6.0,
  'player'

Convert to data frame

In [46]:
import numpy as np
df = pd.DataFrame(seasonGoals)

#manipulate x and y signs to display vertical
df['x'] = np.abs(df['x'].tolist())
df['y'] = (df['y'] * np.sign(df['x'])).tolist()
df

,x,y,player,secondaryType,team,strength,period
0,87.0,-3.0,Artturi Lehkonen,Wrap-around,Montréal Canadiens,Even,1
1,64.0,-14.0,Auston Matthews,Snap Shot,Toronto Maple Leafs,Power Play,1
2,63.0,5.0,John Tavares,Snap Shot,Toronto Maple Leafs,Even,2
3,53.0,7.0,Andrew Shaw,Slap Shot,Montréal Canadiens,Power Play,2
4,74.0,4.0,Auston Matthews,Snap Shot,Toronto Maple Leafs,Even,4
...,...,...,...,...,...,...,...
7753,56.0,-26.0,Matt Roy,Wrist Shot,Los Angeles Kings,Even,2
7754,68.0,3.0,Brandon Pirri,Slap Shot,Vegas Golden Knights,Even,2
7755,85.0,-10.0,Valentin Zykov,Wrist Shot,Vegas Golden Knights,Even,2
7756,81.0,5.0,Anze Kopitar,Snap Shot,Los Angeles Kings,Power Play,3


Download data

In [ ]:
df.to_csv('nhlGoals2018.csv')